In [31]:
from BorderModel import run_Incremental, harmonic_mean
from BorderQuery import select_features, select_mungedata_simple, select_mungedata, insert_predictions
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.grid_search import GridSearchCV
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from dbhelper import PgDB

In [2]:
import os
from ipyparallel import Client
rc = Client()
dview = rc[:]

# set proper working directory on all clients
cwd = os.getcwd()
dview.map(os.chdir, [cwd] * 32)
# print(dview.apply_sync(os.getcwd))

with dview.sync_imports():
    import datetime
    from BorderModel import run_Incremental
    from BorderQuery import select_features, select_mungedata_simple, select_mungedata
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.grid_search import GridSearchCV

importing datetime on engine(s)
importing run_Incremental from BorderModel on engine(s)
importing select_features,select_mungedata_simple,select_mungedata from BorderQuery on engine(s)
importing ExtraTreesRegressor from sklearn.ensemble on engine(s)
importing GridSearchCV from sklearn.grid_search on engine(s)


In [3]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96, max_features='sqrt', max_depth=20)

pr1 = rc[0].apply_async(run_Incremental, model, 3, 1, '2013-1-1', '2016-1-1', '2016-1-1', '2018-1-1')
pr5 = rc[1].apply_async(run_Incremental, model, 3, 5, '2013-1-1', '2016-1-1', '2016-1-1', '2018-1-1')

In [13]:
pr1.ready()

True

## Peace Arch

In [14]:
im1 = pr1.get(1)

In [32]:
ensemble1 = harmonic_mean((im1.y_predict, im1.baseline()), (1, 1))

In [42]:
# for time, val in ensemble1.iteritems():
#     print time, val
#     break
insert_predictions('v2.1', 3, 1, ensemble1.index, ensemble1.values)

## Pacific Highway

In [43]:
im5 = pr5.get(1)

In [44]:
ensemble5 = harmonic_mean((im5.y_predict, im5.baseline()), (1, 1))

In [45]:
insert_predictions('v2.1', 3, 5, ensemble5.index, ensemble5.values)